<a href="https://colab.research.google.com/github/atejap05/pos_data_science_furg/blob/main/disciplinas/Machine_Learning_I/semana06/Furg_ECD_06b_Machine_Learning_I_Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning I - Ensemble Learning
### Prof. Marcelo Malheiros

Código adaptado de Aurélien Geron (licença Apache-2.0)

---

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_:

- NumPy: suporte a vetores, matrizes e operações de Álgebra Linear
- Matplotlib: biblioteca de visualização de dados
- Scikit-Learn: biblioteca com algoritmos de Machine Learning

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sklearn

# Classificação _ensemble_ usando _hard voting_

Neste exemplo usamos novamente o _dataset_ sintético `moons`. E também usamos a função `train_test_split` para dividir aleatoriamente este conjunto em 75% de instâncias para treino e 25% para teste.

In [ ]:
# criação dos dados

from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)

In [ ]:
# separação em conjuntos de treino e de teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.75)

Aqui vamos criar três classificadores usando algoritmos distintos, que em seguida serão combinados em um _ensemble_.

O classificador de votação é construído usando um `VotingClassifier`. Este classificador _ensemble_ funciona da mesma forma que um classificador normal, podendo então ser treinado e gerando previsões.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(random_state=42, solver='lbfgs')
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)
svc_clf = SVC(random_state=42, probability=True, gamma='scale')

In [ ]:
# criação do classificador por hard voting

from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[('log', log_clf), ('rnd', rnd_clf), ('svc', svc_clf)], voting='hard')

# treinamento

voting_clf.fit(X_train, y_train)

Abaixo vamos fazer uma comparação medindo a **acurácia** entre cada um dos três classificadores simples e o classificador _ensemble_ (usando os dados de **teste**, neste caso):

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svc_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(accuracy_score(y_test, y_pred), clf.__class__.__name__)

# Classificação _ensemble_ usando _soft voting_

Os mesmos algoritmos anteriores são usados, pois todos eles geram probabilidades (note que definimos o parâmetro `probability=True` no algoritmo `SVC` explicitamente para ter tais probabilidades).

Então é possível indicar para o `VotingClassifier` que as predições devem ser ponderadas pelas suas respectivas probabilidades, para então gerar a predição final.

In [ ]:
# criação do classificador por soft voting

from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(estimators=[('log', log_clf), ('rnd', rnd_clf), ('svc', svc_clf)], voting='soft')

# treinamento

voting_clf.fit(X_train, y_train)

Novamente fazemos uma comparação medindo a **acurácia** entre cada um dos três classificadores simples e o classificador _ensemble_:

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svc_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(accuracy_score(y_test, y_pred), clf.__class__.__name__)

# Classificação _ensemble_ usando _bagging_

Aqui vamos usar um conjunto de modelos `DecisionTreeClassifier`, do tipo árvore de decisão, combinados em um _ensemble_ usando a técnica de _bagging_.

Note os parâmetros de criação do modelo usando `BaggingClassifier`:

- `n_estimators=500`: cria 500 modelos do tipo árvore de decisão
- `max_samples=100`: cada modelo é treinado com 100 instâncias selecionadas aleatoriamente
- `bootstrap=True`: faz uso de _bagging_ (repetindo instâncias); _pasting_ teria esse parâmetro como `False`

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

# criação do ensemble

bag_clf = BaggingClassifier(DecisionTreeClassifier(), random_state=42,
                            n_estimators=500, max_samples=100, bootstrap=True)

# treinamento
bag_clf.fit(X_train, y_train)

Aqui podemos comparar o resultado de um classificador único, baseado em árvore de descisão, com o _ensemble_ criado com 500 árvores distintas:

In [ ]:
from sklearn.metrics import accuracy_score

# acurácia para modelo simples de árvore de decisão

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree), tree_clf.__class__.__name__)

# acurácia para modelo ensemble

y_pred = bag_clf.predict(X_test)
print(accuracy_score(y_test, y_pred), bag_clf.__class__.__name__)

Vale a pena também visualizar as fronteiras de decisão induzidas tanto pelo classificador simples como pelo _ensemble_, que é feito pelo código abaixo.

In [ ]:
# não se preocupe com este código

from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.45, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    cmap1 = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=cmap1)
    if contour:
        cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], 'yo', alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], 'bs', alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r'$x_1$', fontsize=18)
    plt.ylabel(r'$x_2$', fontsize=18, rotation=0)

fix, axes = plt.subplots(ncols=2, figsize=(14,4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_clf, X, y)
plt.title('árvore de decisão', fontsize=14)
plt.sca(axes[1])
plot_decision_boundary(bag_clf, X, y)
plt.title('árvores de decisão com bagging', fontsize=14)
plt.show()

# Florestas aleatórias

Em geral o algoritmo `RandomForestClassifier` tem todos os hiperparâmetros de um `DecisionTreeClassifier` (para controlar como as árvores crescem) e mais todos os hiperparâmetros de um `BaggingClassifier` (para controlar o uso dos dados de treinamento).

Adicionalmente, o algoritmo `RandomForestClassifier` introduz aleatoriedade extra ao criar árvores; em vez de procurar a melhor _feature_ ao dividir um nodo, este procura a melhor _feature_ entre um subconjunto aleatório de _features_.

Isso resulta em uma maior diversidade de árvores, geralmente resultando em um modelo _ensemble_ ainda melhor.

In [ ]:
# criação, treinamento e predição

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(random_state=42, n_estimators=500, max_leaf_nodes=16)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

# acurácia para modelo simples de árvore de decisão

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree), tree_clf.__class__.__name__)

# acurácia para modelo de floresta aleatória

y_pred = rnd_clf.predict(X_test)
print(accuracy_score(y_test, y_pred), rnd_clf.__class__.__name__)

Para visualizar o funcionamento de uma floresta aleatória, podemos sobrepor as fronteiras de decisão de **n** árvores de decisão, treinadas individualmente.

Quanto mais árvores são mostradas (cada uma parcialmente transparente), mais evidente fica a **fronteira combinada de decisão** deste algoritmo _ensemble_.

In [ ]:
# não se preocupe com os detalhes deste código

n = 1
plt.figure(figsize=(12, 5))
for i in range(n):
    tree_clf = DecisionTreeClassifier(random_state=42 + i, max_leaf_nodes=16)
    indices_with_replacement = np.random.randint(0, len(X_train), len(X_train))
    tree_clf.fit(X[indices_with_replacement], y[indices_with_replacement])
    plot_decision_boundary(tree_clf, X, y, axes=[-1.5, 2.45, -1, 1.5], alpha=0.02, contour=False)
plt.show()

# Importância das _features_

Outra grande utilidade de florestas aleatórias é que estas tornam mais fácil medir a **importância relativa** de cada _feature_.

A importância de uma _feature_ pode ser medida observando quantos nodos das árvores da floresta usam essa _feature_ como critério de decisão. Mais precisamente, se calcula uma média ponderada, em que o peso de cada nodo é igual ao número de instâncias que estão associadas ao mesmo.

A biblioteca `Scikit-Learn` calcula essa pontuação automaticamente para cada _feature_ após o treinamento. Esses resultados são normalizados para que a soma de todas as importâncias seja igual a 1.

Podemos acessar o resultado usando `.feature_importances_`.

O código a seguir treina um algoritmo `RandomForestClassifier` no conjunto de dados IRIS, exibindo a importância de cada _feature_.

No caso, as características mais importantes são o comprimento da pétala (44%) e a largura (42%), enquanto o comprimento e a largura da sépala são pouco importantes em comparação (11% e 2%, respectivamente).

In [ ]:
# leitura dos dados, criação e treinamento do modelo

from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=500)
rnd_clf.fit(iris['data'], iris['target'])

In [ ]:
# lista com as importâncias das features
rnd_clf.feature_importances_

In [ ]:
# features listadas por ordem crescente de importância
for score, name in sorted(zip(rnd_clf.feature_importances_, iris['feature_names'])):
    print('{:06.2%}'.format(score), name)